# Atrial Fibrillation Features

As with activity classification, we will featurize our time series signal and throw those features into a classifier. For AF we will build a two-class classifier using the inter-beat-interval time series. This time series can be derived by taking the difference between successive QRS complex locations as provided by the Pan-Tompkins algorithm in the previous videos. We source our features from a couple submissions to the [Computing in Cardiology Challenge 2017](https://physionet.org/content/challenge-2017/1.0.0/). 

>[Behar, Rosenberg, Yaniv, Oster. Rhythm and Quality Classification from Short ECGs Recorded Using a Mobile Device. Computing in Cardiology Challenge 2017.](http://www.cinc.org/archives/2017/pdf/165-056.pdf)  

and   
>[Bonizzi, Driessens, Karel. Detection of Atrial Fibrillation Episodes from Short Single Lead Recordings by Means of Ensemble Learning. Computing in Cardiology Challenge 2017.](http://www.cinc.org/archives/2017/pdf/169-313.pdf)


Recall that the RR interval time series is irregularly sampled because we get a new measurement at each heart beat, instead of at some fixed interval in time. Many of the features we want can be computed on an irregular time series, but some, especially frequency domain features, cannot. In that case, we will first have to make this uniform by using interpolation.


## Imports

In [3]:
import glob
import os

import numpy as np
import pandas as pd

## Load Data

In [6]:
fs = 300
data_dir = '/data/cinc/'
ref = pd.read_csv(data_dir + 'REFERENCE.csv')
ref = dict(zip(ref.record, ref.rhythm))
base = lambda f: os.path.splitext(os.path.basename(f))[0]
files = sorted(glob.glob(data_dir + '*.npz'))
qrs = []
labels = []
for f in files:
    with np.load(f) as npz:
        qrs.append(npz['qrs'])
    labels.append(ref[base(f)])

## Features

The features for the AF detection algorithm are computed from the RR interval time series. We use the time domain and frequency domain features listed below.

### Time domain
 - minimum RR interval
 - maximum RR interval
 - median RR interval
 - average RR interval
 - standard deviation of RR intervals
 - number of RR interval outliers
   - An RR interval is an outlier if it is greater than 1.2x the average RR interval in the ECG record

 - root-mean-square of the difference between adjacent RR intervals
 - percent of RR interval differences greater than 50 milliseconds

### Frequency domain
The RR interval time series is not sampled regularly in time. We only have a datapoint every heart beat. Before we can compute any frequency domain features, the time series must be resampled so that we get uniform data points. Resample the RR interval time series to 4 Hz before computing the features below.

 - peak magnitude between 0.04 Hz and 0.15 Hz in the regularized RR interval time series
 - main frequency between 0.04 Hz and 0.15 Hz in the regularized RR interval time series
 - peak magnitude between 0.15 Hz and 0.4 Hz in the regularized RR interval time series
 - main frequency between 0.15 Hz and 0.4 Hz in the regularized RR interval time series

In [14]:
def Featurize(qrs_inds, fs):
    """Featurize the qrs complex locations time series.

    Args:
        qrs_inds: (np.array of number) the sample indices of the QRS complex locations
        fs: (number) the sampling rate

    Returns:
        n-tuple of features
    """
    # Compute the RR interval time series
    rr = np.diff(qrs_inds)
    # Compute time domain features
    print(rr)
    min_rr = np.min(rr)
    max_rr = np.max(rr)
    median_rr = np.median(rr)
    mean_rr = np.mean(rr)
    std_rr = np.std(rr)
    n_outliers = np.sum(rr>1.2 * np.mean(rr))
    rmssd = np.sqrt(np.mean(np.square(np.diff(rr))))
    pdrr_50 = np.mean(np.diff(rr)/fs>0.05)

    # Regularly resample the RR interval time series
    rr_ts = np.arange(qrs_inds[0] / fs, qrs_inds[-2] / fs, 1/4)
    rr_interp = np.interp(rr_ts, qrs_inds[:-1] / fs, rr)
    
    # Compute the Fourier transform of the regular RR interval time series
    freq = np.fft.rfftfreq(len(rr_interp), 1/4)
    fft_mag = np.abs(np.fft.rfft(rr_interp))
    
    # Compute frequency domain features
    lf_mag = np.max(fft_mag[(freq >= 0.04) & (freq <= 0.15)])
    lf_freq = freq[np.argmax(fft_mag[(freq >= 0.04) & (freq<= 0.15)])]
    
    hf_mag = np.max(fft_mag[(freq >= 0.15) & (freq <= 0.4)])
    hf_freq = freq[np.argmax(fft_mag[(freq >= 0.15) & (freq<= 0.4)])]    


    return (min_rr, max_rr, median_rr, n_outliers, mean_rr, std_rr, rmssd, pdrr_50,
            lf_mag, lf_freq, hf_mag, hf_freq)

In [7]:
fs

300

In [19]:
feats = [Featurize(qrs_inds, fs)  for qrs_inds in qrs if len(qrs_inds)>0]
X, y = np.array(feats), np.array(labels)

[214 218 237 243 231 239 244 242 234 240 243 239 237 244 246 235 226 230
 229 228 218 215 225 226 232 221 221 222 220 213 219 223 215 212 219 228
 230 234]
[306 301 305 318 305 315 321 308 109 212 325 307 310 313 303 296 302 250
 104  96  60 143  76 101 116 175 290 287 297 310 104 164 298 295 285 299]
[224 232 232 236 237 225 214 212 217 224 231 245 249 246 230 221 217 220
 222 227 232 236 234 233 224 218 211 211 215 222 236 243 242 234 219 213
 210 215 220 228 234 235 230 218 215 206 210 214 211 206 202 209 162  65
 235 233 230 228 225 221 224 128  92 219 218 220 218 222 222 216 210 206
 206 219 230 235 235 229 218 213 224]
[285 277 323 358 310 341 403 254 214 225 254 371 372 236 347 228 238 348
 277 235 218 292 301 231 261 292 262 324 224 222 240]
[295 182 124  96 117 165 132 113 300 186 132 112 123 110 274 248 242 143
 103 128 131 131 130 139 316 200 152 115 113 134 138 141 141 141 142 120
 166 135 110 104 288 178 118 102 111 106 129 320 209 136 109 103 103  98
 109 110 109 110 109 